In [30]:
from utils import open_json, clean_key, save_json, normalize_text
lad_catalog_data = list(filter(lambda i: i['category'] == 'tractor', open_json('lad_catalog_data.json')))
tractordata_catalog = open_json('tractordata_catalog.json')
lad_models = [item['model'] for item in filter(lambda i: i['manufacturer_key'] == 'steyr', lad_catalog_data)]
tractordata_models = [item['model'] for item in filter(lambda i: i['manufacturer_key'] == 'steyr', tractordata_catalog)]


In [31]:
# using nltk
from nltk.tokenize import word_tokenize
import nltk
from fuzzywuzzy import fuzz
from utils import open_json, clean_key, save_json, normalize_text
nltk.download('punkt_tab')
def nltk_fuzzy_similarity(a: str, b: str) -> float:
    normalized_a = normalize_text(a)
    normalized_b = normalize_text(b)

    tokenized_a = word_tokenize(normalized_a)
    tokenized_b = word_tokenize(normalized_b)

    return fuzz.token_sort_ratio(normalized_a, normalized_b) / 100

lad_model = "MULTI 4120"
tractordata_model = "4120 Multi"

similarity = nltk_fuzzy_similarity(lad_model, tractordata_model)


print(f"Similarity: {similarity * 100:.2f} %")

Similarity: 100.00 %


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Lietotajs\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [32]:
# using sentence transformers
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

lad_model = "MULTI 4120"
tractordata_model = "4120 Multi"

def sentence_transformer_similarity(model: SentenceTransformer, a: str, b: str) -> float:
    normalized_a = normalize_text(a)
    normalized_b = normalize_text(b)
    embedding_a = model.encode(normalized_a)
    embedding_b = model.encode(normalized_b)

    return util.cos_sim(embedding_a, embedding_b).item()
similarity = sentence_transformer_similarity(model, lad_model, tractordata_model)
print(f"Similarity: {similarity * 100:.2f} %")

Similarity: 96.49 %


In [33]:
# rapid fuzz
from rapidfuzz import fuzz

lad_model = "MULTI 4120"
tractordata_model = "4120 Multi"
def rapidfuzz_similarity(a: str, b: str) -> float:
    normalized_a = normalize_text(a)
    normalized_b = normalize_text(b)
    return fuzz.token_sort_ratio(normalized_a, normalized_b) / 100

similarity = rapidfuzz_similarity(lad_model, tractordata_model)
print(f"Similarity: {similarity * 100:.2f} %")

Similarity: 100.00 %


In [34]:
# combined sentence transformer & rapid fuzz matching
from rapidfuzz import fuzz
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

def rapidfuzz_similarity(a: str, b: str) -> float:
    normalized_a = normalize_text(a)
    normalized_b = normalize_text(b)
    return fuzz.token_sort_ratio(normalized_a, normalized_b) / 100

def sentence_transformer_similarity(model: SentenceTransformer, a: str, b: str) -> float:
    normalized_a = normalize_text(a)
    normalized_b = normalize_text(b)
    embedding_a = model.encode(normalized_a)
    embedding_b = model.encode(normalized_b)

    return util.cos_sim(embedding_a, embedding_b).item()

# fuzzy and sentence matching
def sentence_transformer_fuzzy_similarity(model: SentenceTransformer, a: str, b: str) -> float: 
    fuzzy_score = rapidfuzz_similarity(a, b)
    sentence_similarity = sentence_transformer_similarity(model, a, b)
    return (fuzzy_score * 0.6) + (sentence_similarity * 0.4), fuzzy_score, sentence_similarity

lad_model = "MULTI 4120"
tractordata_model = "4120 Multi"

similarity, fuzzy, sentence_similarity = sentence_transformer_fuzzy_similarity(model, lad_model, tractordata_model)
print(f"Similarity: {similarity * 100:.2f} %")

Similarity: 98.24 %


In [35]:
from rapidfuzz import fuzz
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

for lmodel in lad_models:
    best_match = (-1, -1, -1, '')
    for tmodel in tractordata_models:
        similarity, fuzzy, sentence_similarity = sentence_transformer_fuzzy_similarity(model, lmodel, tmodel)
        if similarity > best_match[0]:
            best_match = (similarity, fuzzy, sentence_similarity , tmodel)
    print(f"'{lmodel}' best match '{best_match[3]}' with similarity {best_match[0]} (fuzzy: {best_match[1]}, sentence: {best_match[2]})")

'MULTI 4120' best match '4120 Multi' with similarity 0.9824487268924713 (fuzzy: 1.0, sentence: 0.9648974537849426)
'MULTI 4120' best match '4120 Multi' with similarity 0.9824487268924713 (fuzzy: 1.0, sentence: 0.9648974537849426)
'KOMPAKT 4100' best match '4100 Multi' with similarity 0.6419656330888921 (fuzzy: 0.6363636363636364, sentence: 0.647567629814148)
'KOMPAKT 4080' best match '4100 Multi' with similarity 0.4719266959212043 (fuzzy: 0.5454545454545454, sentence: 0.39839884638786316)
'PROFI 6150' best match '6140 Profi' with similarity 0.8886162459850311 (fuzzy: 0.9, sentence: 0.8772324919700623)
'PROFI 6150 CVT' best match '4120 Profi CVT' with similarity 0.82660254410335 (fuzzy: 0.8571428571428572, sentence: 0.7960622310638428)
'IMPULS 6165 CVT' best match '6150 CVT' with similarity 0.7168168853158536 (fuzzy: 0.6086956521739131, sentence: 0.8249381184577942)
'IMPULS 6175 CVT' best match '6175 CVT' with similarity 0.7614860767903535 (fuzzy: 0.6956521739130435, sentence: 0.8273199

In [4]:
x = {'category': 'tractor', 'sub_category': 'tractor_4x4', 'manufacturer': 'KUBOTA', 'model': 'L1-382', 'sources': ['https://eps.lad.gov.lv/show_etk_public?action=index&controller=catalog_entries_public&id=11555&page=39'], 'price': 32600.0, 'equipment_level_code': 'medium', 'specification': {
    'powertrain': '4x4', 
    'engine_power_kw': 28.3366, 
    'engine_cylinders': 3, 
    'hydrolic_pump_flow': 45.5, 
    'lift_capacity': 906.0
    }, 'model_key': 'l1382', 'manufacturer_key': 'kubota'} 

y = {'category': 'tractor', 'sub_category': 'tractor_4x4', 'manufacturer': 'Kubota', 'model': 'L1-325', 'sources': ['https://www.tractordata.com/farm-tractors/009/6/4/9640-kubota-l1-325.html'], 'price': -1, 'equipment_level_code': 'base', 'specification': {
    'engine_power_kw': 23.6, 
    'pto_power_kw': 22.0, 
    'powertrain': '4x4', 
    'hydrolic_pump_flow': 23.5, 
    'lift_capacity': 729.0, 
    'wheelbase': 1.73, 
    'weight': 1389.0
    }, 'model_key': 'l1325', 'manufacturer_key': 'kubota'}

print({**y['specification'], **x['specification']})

{'engine_power_kw': 28.3366, 'pto_power_kw': 22.0, 'powertrain': '4x4', 'hydrolic_pump_flow': 45.5, 'lift_capacity': 906.0, 'wheelbase': 1.73, 'weight': 1389.0, 'engine_cylinders': 3}


In [ ]:
from utils import save_json
data = """tractor_4x2;0.10;0.20;0.30;0.20;0.30;0.40;0.50;0.60;0.70;0.80;0.90;1.00;1.10;1.20;1.30;1.40;1.50;1.50;1.50;1.50;1.50;3.00;3.25;3.50;3.75;4.25;4.50;4.75;5.10;5.40;5.80;6;11;18;25;34;45;57;70;;
tractor_4x4;0;0;0;0;0;0;0;0;0;0;0;0.00;0.10;0.20;0.30;0.40;0.60;0.60;0.70;0.80;0.90;1.00;1.10;1.20;1.30;1.40;1.50;1.50;1.50;1.50;1.50;3;5;8;11;15;19;24;30;;
plough;0.50;0.75;1.00;2;4;6;9;12;16;19;24;29;35;40;47;53;61;68;76;84;93;101;;;;;;;;;;;;;;;;;;;
harrow;1.00;1.25;1.50;3;5;7;10;13;17;20;24;27;31;35;39;43;48;52;56;61;66;71;;;;;;;;;;;;;;;;;;;
cultivator;1.00;1.25;1.50;3;5;7;10;13;17;20;24;27;31;35;39;43;48;52;56;61;66;71;;;;;;;;;;;;;;;;;;;
row_cultivator;0.00;0.00;0.00;0;1;2;4;6;8;10;14;17;21;25;31;36;42;48;55;62;70;78;;;;;;;;;;;;;;;;;;;
packing_press;0.00;0.25;0.50;1;3;4;7;9;12;15;19;23;28;32;37;42;48;54;60;66;73;80;;;;;;;;;;;;;;;;;;;
balling_press;0.50;0.75;1;2;5;8;12;17;23;29;36;43;46.10;50.36;54.62;58.88;63.14;67.40;71.66;75.91;80.17;84.43;;;;;;;;;;;;;;;;;;;
combine;0;0;0;0;0;0.30;0.70;1;1.30;1.70;2;3.00;3.50;4;4.60;5.40;6;7;8;9;10;11;12;14.00;15.00;16;18.00;19.00;20;22.50;23.80;25;27.50;28.70;30;32.50;33.80;35;37.50;38.80;40
potato_combine;0.50;0.75;1.00;2;4;5;7;9;12;14;17;19;22;25;28;30;34;37;40;43;47;50;;;;;;;;;;;;;;;;;;;
seed_drill;0.00;0.00;0;1;3;5;7;11;15;20;26;32;32.91;36;39;42;45;48;52;55;58;61;;;;;;;;;;;;;;;;;;;
planter;0.00;0.00;0;1;3;5;7;11;15;20;26;32;32.91;36.03;39.14;42.26;45.37;48.49;51.60;54.72;57.83;60.95;;;;;;;;;;;;;;;;;;;
"""
result = {}
for line in data.splitlines():
    values = line.split(';')
    result[values[0]] = list(map(lambda v: round(float('0' if len(v) <= 0 else v) / 100, 2), values[1:]))
save_json("repair_cost.json", result)